In [1]:
import pandas as pd
import numpy as np
from Bio import Seq, SeqIO
from Bio import pairwise2

In [2]:
js1 = pd.read_json('./01-Pfizer_BNT-162b2.json',orient='index')[0].to_dict()
js2 = pd.read_json('./02-Moderna_mRNA-1273.json',orient='index')[0].to_dict()

In [3]:
s1 = Seq.Seq(js1['c1'])
s2 = Seq.Seq(js2['c2'])

In [4]:
spike1 = s1.translate()
spike2 = s2.translate()

# mutaions
## https://www.uniprot.org/uniprot/P59594

In [5]:
muts = {'D614G':'Enhance the infection ability', 
       'C323A':'No effect on human ACE2 binding in vitro', 
       'E452A':'90% loss of human ACE2 binding in vitro'}

In [6]:
s1_muts = []
s1_muts.append(s1) #orignal

for i,m in zip([610, 319, 448], ['GGT', 'GCC', 'GCG']):
    print(spike1[i], s1[i*3:(i+1)*3])
    s = s1.tomutable()
    s[i*3:(i+1)*3] = m
    s1_muts.append(s.toseq())

D GAT
C TGC
E GAG


In [7]:
s2_muts = []

s2_muts.append(s2) #orignal
for i,m in zip([610, 319, 448], ['GGT', 'GCC', 'GCG']):
    print(spike2[i], s2[i*3:(i+1)*3])
    s = s2.tomutable()
    s[i*3:(i+1)*3] = m
    s2_muts.append(s.toseq())

D GAC
C TGC
E GAG


In [8]:
fs1=[]
for s1m in s1_muts:
    mrna1 = Seq.Seq('').join([Seq.Seq(js1['h1']), s1m, Seq.Seq(js1['t1'])])
    spike1 = s1m.translate()
    fs1.append([mrna1, s1m, spike1])
df1 = pd.DataFrame(fs1, index=['orignal', 'D614G', 'C323A', 'E452A'], columns=['mRNA_full', 'mRNA-spike', 'spike-protein'])
df1

,mRNA_full,mRNA-spike,spike-protein
orignal,"(G, A, G, A, A, T, A, A, A, C, T, A, G, T, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, A, G, A, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."
D614G,"(G, A, G, A, A, T, A, A, A, C, T, A, G, T, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, A, G, A, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."
C323A,"(G, A, G, A, A, T, A, A, A, C, T, A, G, T, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, A, G, A, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."
E452A,"(G, A, G, A, A, T, A, A, A, C, T, A, G, T, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, A, G, A, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."


In [9]:
fs2=[]
for s2m in s2_muts:
    mrna2 = Seq.Seq('').join([Seq.Seq(js2['h2']), s2m, Seq.Seq(js2['t2'])])
    spike2 = s2m.translate()
    fs2.append([mrna2, s2m, spike2])
df2 = pd.DataFrame(fs2, index=['orignal', 'D614G', 'C323A', 'E452A'], columns=['mRNA_full', 'mRNA-spike', 'spike-protein'])
df2

,mRNA_full,mRNA-spike,spike-protein
orignal,"(G, G, G, A, A, A, T, A, A, G, A, G, A, G, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, C, G, G, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."
D614G,"(G, G, G, A, A, A, T, A, A, G, A, G, A, G, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, C, G, G, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."
C323A,"(G, G, G, A, A, A, T, A, A, G, A, G, A, G, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, C, G, G, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."
E452A,"(G, G, G, A, A, A, T, A, A, G, A, G, A, G, A, ...","(A, A, C, C, T, G, A, C, C, A, C, C, C, G, G, ...","(N, L, T, T, R, T, Q, L, P, P, A, Y, T, N, S, ..."


In [10]:
df1.to_csv('./bnt.csv')
df2.to_csv('./mod.csv')